In [1]:
# 导入预训练的vgg
import torch
import torchsummary as summary
import net
import function
import torch.nn as nn
import torchvision.models as models
vgg = models.vgg16(pretrained=True)

OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
decoder = net.decoder
encoder = nn.Sequential(*list(vgg.features.children())[:-1])
net=net.Net(encoder,decoder).cuda()
net.train()

In [ ]:
from torchvision import transforms
def train_transform():
    transform_list = [
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor()
    ]
    return transforms.Compose(transform_list)

In [ ]:
tf = train_transform()

In [ ]:
import torchvision
from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path
import PIL.Image as Image
# 定义一个自定义的数据集类，继承自Dataset类
class MyDataset(Dataset):
# 初始化函数，接受一个文件夹路径和一个变换函数作为参数
    def __init__(self, folder_path, transform=None):
        # 初始化文件路径
        self.path=folder_path
        self.paths=list(Path(folder_path).glob('*.jpg'))
        # 保存变换函数
        self.transform = transform

    # 定义数据集的长度，即文件的个数
    def __len__(self):
        return len(self.paths)

    # 定义数据集的索引方式，即如何根据索引返回一个数据
    def __getitem__(self, index):
        # 根据索引获取文件名
        path = self.paths[index]
        # 读取图像数据并返回
        data = Image.open(path)
        if self.transform:
            data = self.transform(data)
        return data
content_dataset = MyDataset('content/images',tf)
style_dataset = MyDataset('style/images',tf)
content_iter = torch.utils.data.DataLoader(content_dataset, batch_size=16,shuffle=True)
style_iter = torch.utils.data.DataLoader(style_dataset, batch_size=16, shuffle=True)
# adam optim
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
iter_count = 1000
weight_content = 1
weight_style = 20
for i in tqdm(range(iter_count)):
    content = next(iter(content_iter)).to('cuda')
    style = next(iter(style_iter)).to('cuda')
    loss_content, loss_style, generate_img = net(content, style)
    loss = loss_content * weight_content + loss_style * weight_style
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
        print('第{}次迭代，content损失为：{:.3f}，style损失为：{:.3f}'.format(i + 1, loss_content, loss_style))
        # 保存生成的图像
        torchvision.utils.save_image(generate_img, 'result-{}.jpg'.format(i + 1))

    if (i + 1) % 100 == 0:
        # 保存模型
        torch.save(net.state_dict(), 'model.pth')


[WindowsPath('content/images/n0153282900000005.jpg'),
 WindowsPath('content/images/n0153282900000006.jpg'),
 WindowsPath('content/images/n0153282900000007.jpg'),
 WindowsPath('content/images/n0153282900000010.jpg'),
 WindowsPath('content/images/n0153282900000014.jpg'),
 WindowsPath('content/images/n0153282900000015.jpg'),
 WindowsPath('content/images/n0153282900000016.jpg'),
 WindowsPath('content/images/n0153282900000017.jpg'),
 WindowsPath('content/images/n0153282900000018.jpg'),
 WindowsPath('content/images/n0153282900000019.jpg'),
 WindowsPath('content/images/n0153282900000020.jpg'),
 WindowsPath('content/images/n0153282900000021.jpg'),
 WindowsPath('content/images/n0153282900000023.jpg'),
 WindowsPath('content/images/n0153282900000028.jpg'),
 WindowsPath('content/images/n0153282900000031.jpg'),
 WindowsPath('content/images/n0153282900000033.jpg'),
 WindowsPath('content/images/n0153282900000034.jpg'),
 WindowsPath('content/images/n0153282900000036.jpg'),
 WindowsPath('content/images